In [1]:
# import library
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.read_csv('../data/feature_selection_positive.csv')


# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [3]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_pos_fs.sav', 'wb'))

In [4]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_pos_fs.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.29107
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.2112
[3]	valid_0's multi_logloss: 1.14454
[4]	valid_0's multi_logloss: 1.0836
[5]	valid_0's multi_logloss: 1.02871
[6]	valid_0's multi_logloss: 0.982166
[7]	valid_0's multi_logloss: 0.939255
[8]	valid_0's multi_logloss: 0.903622
[9]	valid_0's multi_logloss: 0.87048
[10]	valid_0's multi_logloss: 0.840046
[11]	valid_0's multi_logloss: 0.811601
[12]	valid_0's multi_logloss: 0.785023
[13]	valid_0's multi_logloss: 0.76353
[14]	valid_0's multi_logloss: 0.740205
[15]	valid_0's multi_logloss: 0.717713
[16]	valid_0's multi_logloss: 0.699931
[17]	valid_0's multi_logloss: 0.683468
[18]	valid_0's multi_logloss: 0.668712
[19]	valid_0's multi_logloss: 0.653632
[20]	valid_0's multi_logloss: 0.637297
[21]	valid_0's multi_logloss: 0.625027
[22]	valid_0's multi_logloss: 0.611079
[23]	valid_0's multi_logloss: 0.599969
[24]	valid_0's multi_logloss: 0.588223
[25]	valid_0's multi_

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [5]:
# set parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [6]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
pickle.dump(model, open('../model/xgb_pos_fs.sav', 'wb'))
t.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_fs.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 1s 1ms/step - loss: 1.8171 - acc: 0.2560 - val_loss: 1.3786 - val_acc: 0.2500
Epoch 2/100
1340/1340 [==============================] - 0s 94us/step - loss: 1.7592 - acc: 0.2716 - val_loss: 1.3361 - val_acc: 0.3155
Epoch 3/100
1340/1340 [==============================] - 0s 84us/step - loss: 1.7310 - acc: 0.2746 - val_loss: 1.3156 - val_acc: 0.3185
Epoch 4/100
1340/1340 [==============================] - 0s 89us/step - loss: 1.6971 - acc: 0.2709 - val_loss: 1.3094 - val_acc: 0.3780
Epoch 5/100
1340/1340 [==============================] - 0s 82us/step - loss: 1.5294 - acc: 0.3067 - val_loss: 1.3121 - val_acc: 0.3720
Epoch 6/100
1340/1340 [==============================] - 0s 78us/step - loss: 1.5551 - acc: 0.3149 - val_loss: 1.3157 - val_acc: 0.3958
Epoch 7/100
1340/1340 [==============================] - 0s 87us/step - loss: 1.5082 - acc: 0.3433 - val_loss: 1.3164 - val_acc: 0.3720
Ep

1340/1340 [==============================] - 0s 82us/step - loss: 0.6510 - acc: 0.7269 - val_loss: 0.8600 - val_acc: 0.6518
Epoch 61/100
1340/1340 [==============================] - 0s 80us/step - loss: 0.6545 - acc: 0.7201 - val_loss: 0.8578 - val_acc: 0.6696
Epoch 62/100
1340/1340 [==============================] - 0s 81us/step - loss: 0.6332 - acc: 0.7366 - val_loss: 0.8651 - val_acc: 0.6577
Epoch 63/100
1340/1340 [==============================] - 0s 79us/step - loss: 0.5977 - acc: 0.7567 - val_loss: 0.8760 - val_acc: 0.6845
Epoch 64/100
1340/1340 [==============================] - 0s 80us/step - loss: 0.6218 - acc: 0.7448 - val_loss: 0.8703 - val_acc: 0.6696
Epoch 65/100
1340/1340 [==============================] - 0s 83us/step - loss: 0.5851 - acc: 0.7687 - val_loss: 0.8630 - val_acc: 0.6637
Epoch 66/100
1340/1340 [==============================] - 0s 89us/step - loss: 0.6266 - acc: 0.7343 - val_loss: 0.8649 - val_acc: 0.6339
Epoch 67/100
1340/1340 [==============================

In [8]:
f = []
f.append(t)

In [9]:
f

[[75.0, 84.82, 80.95, 64.88]]

In [10]:
#read data
df = pd.read_csv('../data/feature_selection_negative.csv')

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [11]:
# define and fit 
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
print(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_ng_fs.sav', 'wb'))

73.4


In [12]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

# pickle.dump(gbm, open('../model/lgbm_ng_fs.sav', 'wb'))
# result append to list
t.append(
round(gbm.score(X_test, y_test)*100, 2)
)
pickle.dump(gbm, open('../model/lgbm_ng_fs.sav', 'wb'))

[1]	valid_0's multi_logloss: 1.31034
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.24189
[3]	valid_0's multi_logloss: 1.18161
[4]	valid_0's multi_logloss: 1.1357
[5]	valid_0's multi_logloss: 1.08979
[6]	valid_0's multi_logloss: 1.0541
[7]	valid_0's multi_logloss: 1.01416
[8]	valid_0's multi_logloss: 0.980564
[9]	valid_0's multi_logloss: 0.952559
[10]	valid_0's multi_logloss: 0.920911
[11]	valid_0's multi_logloss: 0.898194
[12]	valid_0's multi_logloss: 0.880214
[13]	valid_0's multi_logloss: 0.856074
[14]	valid_0's multi_logloss: 0.837627
[15]	valid_0's multi_logloss: 0.820634
[16]	valid_0's multi_logloss: 0.803493
[17]	valid_0's multi_logloss: 0.787869
[18]	valid_0's multi_logloss: 0.777264
[19]	valid_0's multi_logloss: 0.76643
[20]	valid_0's multi_logloss: 0.75791
[21]	valid_0's multi_logloss: 0.748808
[22]	valid_0's multi_logloss: 0.7387
[23]	valid_0's multi_logloss: 0.732538
[24]	valid_0's multi_logloss: 0.724325
[25]	valid_0's multi_logl

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [14]:
# define and fit 
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
t.append(round(model.score(X_test, y_test)*100, 2))
pickle.dump(model, open('../model/xgb_ng_fs.sav', 'wb'))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_fs.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 1s 3ms/step - loss: 2.2255 - acc: 0.2252 - val_loss: 1.5107 - val_acc: 0.4468
Epoch 2/100
373/373 [==============================] - 0s 55us/step - loss: 1.9330 - acc: 0.2815 - val_loss: 1.5329 - val_acc: 0.4787
Epoch 3/100
373/373 [==============================] - 0s 43us/step - loss: 1.7776 - acc: 0.3003 - val_loss: 1.5756 - val_acc: 0.4894
Epoch 4/100
373/373 [==============================] - 0s 46us/step - loss: 1.5834 - acc: 0.3968 - val_loss: 1.6402 - val_acc: 0.5106
Epoch 5/100
373/373 [==============================] - 0s 43us/step - loss: 1.6744 - acc: 0.3566 - val_loss: 1.6332 - val_acc: 0.5106
Epoch 6/100
373/373 [==============================] - 0s 41us/step - loss: 1.7131 - acc: 0.4075 - val_loss: 1.5706 - val_acc: 0.5106
Epoch 7/100
373/373 [==============================] - 0s 42us/step - loss: 1.6863 - acc: 0.3914 - val_loss: 1.4825 - val_acc: 0.5106
Epoch 8/100
373/37

In [16]:
f.append(t)

In [17]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index=['Feature_Selection_Positive', 'Feature_Selection_Negative']
)

,Random Forest,LightGBM,XGBoost,Keras
Feature_Selection_Positive,75.0,84.82,80.95,64.88
Feature_Selection_Negative,73.4,72.34,73.40,62.77


In [18]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index=['Feature_Selection_Positive', 'Feature_Selection_Negative']
).to_csv('../result/Feature_Selection_result.csv')